In [ ]:
import pandas as pd
from datetime import datetime
import json
from pprint import pprint

with open('data.json') as data_file:    
    data = json.load(data_file)

user_data_df=pd.DataFrame(data)
# user_data_df.head()

#user_data_df["last_trip_date"=="2014-06-17"]
user_data_df
user_data_df.to_excel("user_data.xlsx", encoding='utf-8')

# Data description:
# city: city this user signed up in
# phone: primary device for this user
# signup_date: date of account registration; in the form ‘YYYY-MM-DD’
# last_trip_date: the last time this user completed a trip; in the form ‘YYYY-MM-DD’
# avg_dist: the average distance (in miles) per trip taken in the first 30 days after signup
# avg_rating_by_driver: the rider’s average rating over all of their trips
# avg_rating_of_driver: the rider’s average rating of their drivers over all of their trips
# surge_pct: the percent of trips taken with surge multiplier > 1
# avg_surge: The average surge multiplier over all of this user’s trips
# trips_in_first_30_days: the number of trips this user took in the first 30 days after signing up
# uber_black_user: TRUE if the user took an Uber Black in their first 30 days; FALSE otherwise
# weekday_pct: the percent of the user’s trips occurring during a weekday
    
user_data=user_data_df.sort_values(by='last_trip_date')
user_data.tail(40)

# count = 0
# for row in user_data_df.rows:
#     if

In [ ]:
user_data.describe()

In [ ]:
print(user_data['city'].value_counts())
print('\n')
print(user_data['phone'].value_counts())


In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

plt.hist(user_data["weekday_pct"])

In [ ]:
user_data.boxplot(column='avg_rating_by_driver')

In [ ]:
##Pre-processing the data
##Converting categorial variables

from sklearn.preprocessing import LabelEncoder
var_mod = ['uber_black_user']
le = LabelEncoder()
for i in var_mod:
    user_data[i] = le.fit_transform(user_data[i])
    
user_data.dtypes

In [ ]:
# user_data.loc[0:100]['Active_User']=1
# user_data.loc[101:49999]['Active_User']=0

#user_data.describe()

user_data['Active_User'] = np.where(user_data['avg_dist']>3, 1, 0)

user_data.tail(10)

In [ ]:
##Model
import numpy as np

#Import models from scikit learn module:
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import KFold   #For K-fold cross validation
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn import metrics

#Generic function for making a classification model and accessing performance:
def classification_model(model, data, predictors, outcome):

    model.fit(data[predictors],data[outcome])
    predictions = model.predict(data[predictors])
    accuracy = metrics.accuracy_score(predictions,data[outcome])
    print("Accuracy : %s" % "{0:.3%}".format(accuracy))
    kf = KFold(data.shape[0], n_folds=5)
    error = []
    
    for train, test in kf:
        train_predictors = (data[predictors].iloc[train,:])
        train_target = data[outcome].iloc[train]
        model.fit(train_predictors, train_target)
        error.append(model.score(data[predictors].iloc[test,:], data[outcome].iloc[test]))

    print("Cross-Validation Score : %s" % "{0:.3%}".format(np.mean(error)))
    model.fit(data[predictors],data[outcome]) 



In [ ]:
outcome_var = 'Active_User'
model = LogisticRegression()
predictor_var = ['avg_dist']
classification_model(model, user_data, predictor_var, outcome_var)